In [1]:
from pathlib import Path
from src.inspector_git.linker.models import GitProject
from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader
from src.run_blame import check_blame
import sys
# sys.setrecursionlimit(16336)

def build_and_check_project(iglog_path: str, repo_path: str, compute_annotated_lines :bool) -> GitProject:
    iglog_file = Path(iglog_path)

    with open(iglog_file, "r", encoding="utf-8") as f:
        git_log_dto = IGLogReader().read(f)

    transformer = GitProjectTransformer(
        git_log_dto,
        name=iglog_file.stem,
        compute_annotated_lines=compute_annotated_lines,
    )
    project = transformer.transform()

    if compute_annotated_lines:
        check_blame(project, repo_path)

    return project

project = build_and_check_project(
    "../../test-input/inspector-git/TestInspectorGitRepo.iglog",
    "/home/vortex/Work/BachelorThesis/voyager-target/zeppelin/zeppelin",
    False
)
import pickle

with open("project.pkl", "wb") as f:
    pickle.dump(project, f)

print("GitProject saved to project.pkl")

with open("project.pkl", "rb") as f:
    loaded_project = pickle.load(f)

2025-09-21 00:27:57,790 [INFO] src.inspector_git.linker.transformers: Creating GIT project TestInspectorGitRepo
2025-09-21 00:27:57,791 [INFO] src.inspector_git.linker.transformers: Creating commit 1 / 8 (12%)
2025-09-21 00:27:57,793 [INFO] src.inspector_git.linker.transformers: Creating commit 2 / 8 (25%)
2025-09-21 00:27:57,793 [INFO] src.inspector_git.linker.transformers: Creating commit 3 / 8 (37%)
2025-09-21 00:27:57,794 [INFO] src.inspector_git.linker.transformers: Creating commit 4 / 8 (50%)
2025-09-21 00:27:57,794 [INFO] src.inspector_git.linker.transformers: Creating commit 5 / 8 (62%)
2025-09-21 00:27:57,795 [INFO] src.inspector_git.linker.transformers: Creating commit 6 / 8 (75%)
2025-09-21 00:27:57,795 [INFO] src.inspector_git.linker.transformers: Creating commit 7 / 8 (87%)
2025-09-21 00:27:57,796 [INFO] src.inspector_git.linker.transformers: Creating commit 8 / 8 (100%)
2025-09-21 00:27:57,796 [INFO] src.inspector_git.linker.transformers: Done creating GIT project TestIns

GitProject saved to project.pkl
